# Portfolio Optimization Project

### Importing the necessary packages into Python.

In [142]:
#importing packages...there are some extra packages not used.
from pandas_datareader import data, wb
import quandl as qd
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
import chart_studio.plotly as py
import pulp
%matplotlib inline

#import package for optimization algorithm
from scipy.optimize import minimize

### Scraping S&P 500 stock data to get tickers for the top 20 companies by market cap.

In [143]:
sp500_data = pd.read_html("https://stockmarketmba.com/stocksinthesp500.php",
                                 header=0)[0]
sp500_data['Market cap'][3]
for index in range(len(sp500_data['Market cap'])):
    #removing the dollar sign...or the first character in string
    sp500_data['Market cap'][index]=sp500_data['Market cap'][index][1:]
    
    #removing commas
    sp500_data['Market cap'][index]=sp500_data['Market cap'][index].replace(',','')
    
    #changing data type to integer
    sp500_data['Market cap'][index]=float(sp500_data['Market cap'][index])
    sp500_data['Market cap'][index]=int(sp500_data['Market cap'][index])

C:\Users\chait\AppData\Local\Temp\ipykernel_1868\147038754.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data['Market cap'][index]=sp500_data['Market cap'][index][1:]
C:\Users\chait\AppData\Local\Temp\ipykernel_1868\147038754.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data['Market cap'][index]=sp500_data['Market cap'][index].replace(',','')
C:\Users\chait\AppData\Local\Temp\ipykernel_1868\147038754.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [144]:
sp500_data['Market cap'][1]

9092113409

In [146]:
sp500_data.head()

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,A,Agilent Technologies Inc,Common stocks,Large cap,Health Care,45035518004,0.56%,0.0,0.0,0.0,Analyze
1,AAL,American Airlines Group Inc,Common stocks,Mid cap,Industrials,9092113409,0.00%,0.0,0.0,0.0,Analyze
2,AAP,Advance Auto Parts Inc,Common stocks,Mid cap,Consumer Discretionary,9207709485,3.86%,0.0,0.0,0.0,Analyze
3,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,1988832912360,0.73%,0.0,0.0,0.0,Analyze
4,ABBV,Abbvie Inc,Common stocks,Large cap,Health Care,289128878253,3.40%,0.0,0.0,0.0,Analyze


In [147]:
#getting top 20 stocks by market cap of company
top20 = sp500_data.sort_values(by='Market cap',ascending=False)[1:24]
top20.head()

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
3,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,1988832912360,0.73%,0.0,0.0,0.0,Analyze
318,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,1657655067218,1.14%,0.0,0.0,0.0,Analyze
205,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,1042954820000,0.00%,0.0,0.0,0.0,Analyze
206,GOOGL,Alphabet Inc Class A,Common stocks,Large cap,Communication Services,1042954820000,0.00%,0.0,0.0,0.0,Analyze
34,AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,847961495109,0.00%,0.0,0.0,0.0,Analyze


In [148]:
#defining variable that stores ticker symbols of top 20 companies
tickers = top20['Symbol']
tickers = tickers.tolist()


#adding 'WIKI/' to strings for passing into quandl
for x in range(len(tickers)):
    tickers[x] = 'WIKI/'+tickers[x]
    tickers[x] = tickers[x].replace('.','_')
tickers.remove('WIKI/META')
tickers.remove('WIKI/GOOG')
tickers.remove('WIKI/ABBV')
tickers

['WIKI/AAPL',
 'WIKI/MSFT',
 'WIKI/GOOGL',
 'WIKI/AMZN',
 'WIKI/BRK_B',
 'WIKI/JNJ',
 'WIKI/UNH',
 'WIKI/XOM',
 'WIKI/V',
 'WIKI/JPM',
 'WIKI/WMT',
 'WIKI/PG',
 'WIKI/NVDA',
 'WIKI/TSLA',
 'WIKI/LLY',
 'WIKI/CVX',
 'WIKI/MA',
 'WIKI/HD',
 'WIKI/MRK',
 'WIKI/PFE']

### Creating a dataframe based on the tickers we got in the previous step.

In [149]:
qd.ApiConfig.api_key = "Your API Key"
#making empty list to pass stock data
empty = []

#passing data with for loop
for x in range(len(tickers)):
    empty.append(qd.get(tickers[x], start_date="2013-01-01", end_date="2023-01-01"))

#concatenate the data
df = pd.concat(empty,axis=1,keys=tickers)
df.head()

WIKI/AAPL                                                        \
                Open      High       Low     Close      Volume Ex-Dividend   
Date                                                                         
2013-01-02   553.820  555.0000  541.6300  549.0300  20018500.0         0.0   
2013-01-03   547.880  549.6700  541.0000  542.0959  12605900.0         0.0   
2013-01-04   536.965  538.6299  525.8286  527.0000  21226200.0         0.0   
2013-01-07   522.000  529.3000  515.2000  523.9000  17291300.0         0.0   
2013-01-08   529.210  531.8900  521.2500  525.3100  16382400.0         0.0   

                                                         ... WIKI/PFE         \
           Split Ratio  Adj. Open  Adj. High   Adj. Low  ...      Low  Close   
Date                                                     ...                   
2013-01-02         1.0  71.816894  71.969911  70.236149  ...    25.33  25.91   
2013-01-03         1.0  71.046621  71.278740  70.154453  ...    25.56  25.85   
2013-01-04         1.0  69.631213  69.847109  68.187094  ...    25.76  25.96   
2013-01-07         1.0  67.690619  68.637250  66.808825  ...    25.77  25.98   
2013-01-08         1.0  68.625579  68.973109  67.593362  ...    25.98  26.02   

                                                                      \
                Volume Ex-Dividend Split Ratio  Adj. Open  Adj. High   
Date                                                                   
2013-01-02  33522200.0         0.0         1.0  21.396763  21.715242   
2013-01-03  33757400.0         0.0         1.0  21.773909  21.782290   
2013-01-04  28604900.0         0.0         1.0  21.664955  21.782290   
2013-01-07  25779900.0         0.0         1.0  21.715242  21.832576   
2013-01-08  31057000.0         0.0         1.0  21.790671  22.000197   

                                              
             Adj. Low Adj. Close Adj. Volume  
Date                                          
2013-01-02  21.229142  21.715242  33522200.0  
2013-01-03  21.421906  21.664955  33757400.0  
2013-01-04  21.589526  21.757147  28604900.0  
2013-01-07  21.597907  21.773909  25779900.0  
2013-01-08  21.773909  21.807433  31057000.0  

[5 rows x 240 columns]

In [150]:
#set column names for indexing
df.columns.names = ['Tickers','Stock Info']
df.head()

Tickers    WIKI/AAPL                                                        \
Stock Info      Open      High       Low     Close      Volume Ex-Dividend   
Date                                                                         
2013-01-02   553.820  555.0000  541.6300  549.0300  20018500.0         0.0   
2013-01-03   547.880  549.6700  541.0000  542.0959  12605900.0         0.0   
2013-01-04   536.965  538.6299  525.8286  527.0000  21226200.0         0.0   
2013-01-07   522.000  529.3000  515.2000  523.9000  17291300.0         0.0   
2013-01-08   529.210  531.8900  521.2500  525.3100  16382400.0         0.0   

Tickers                                                  ... WIKI/PFE         \
Stock Info Split Ratio  Adj. Open  Adj. High   Adj. Low  ...      Low  Close   
Date                                                     ...                   
2013-01-02         1.0  71.816894  71.969911  70.236149  ...    25.33  25.91   
2013-01-03         1.0  71.046621  71.278740  70.154453  ...    25.56  25.85   
2013-01-04         1.0  69.631213  69.847109  68.187094  ...    25.76  25.96   
2013-01-07         1.0  67.690619  68.637250  66.808825  ...    25.77  25.98   
2013-01-08         1.0  68.625579  68.973109  67.593362  ...    25.98  26.02   

Tickers                                                               \
Stock Info      Volume Ex-Dividend Split Ratio  Adj. Open  Adj. High   
Date                                                                   
2013-01-02  33522200.0         0.0         1.0  21.396763  21.715242   
2013-01-03  33757400.0         0.0         1.0  21.773909  21.782290   
2013-01-04  28604900.0         0.0         1.0  21.664955  21.782290   
2013-01-07  25779900.0         0.0         1.0  21.715242  21.832576   
2013-01-08  31057000.0         0.0         1.0  21.790671  22.000197   

Tickers                                       
Stock Info   Adj. Low Adj. Close Adj. Volume  
Date                                          
2013-01-02  21.229142  21.715242  33522200.0  
2013-01-03  21.421906  21.664955  33757400.0  
2013-01-04  21.589526  21.757147  28604900.0  
2013-01-07  21.597907  21.773909  25779900.0  
2013-01-08  21.773909  21.807433  31057000.0  

[5 rows x 240 columns]

In [151]:
#max closing price from past 10 years
df.xs(key='Close',axis=1,level='Stock Info').max()

Tickers
WIKI/AAPL      647.35
WIKI/MSFT       96.77
WIKI/GOOGL    1220.17
WIKI/AMZN     1598.39
WIKI/BRK_B     217.25
WIKI/JNJ       148.14
WIKI/UNH       248.47
WIKI/XOM       104.38
WIKI/V         278.29
WIKI/JPM       118.77
WIKI/WMT       109.55
WIKI/PG         94.40
WIKI/NVDA      250.48
WIKI/TSLA      385.00
WIKI/LLY        89.98
WIKI/CVX       134.85
WIKI/MA        843.60
WIKI/HD        207.23
WIKI/MRK        66.58
WIKI/PFE        39.02
dtype: float64

### Extracting the adj. returns columns and creating dataframes for the arithmetic/logarthmic returns.

In [153]:
#extracting adj. close section from df
adjclose = df.xs(key='Adj. Close',axis=1,level='Stock Info')
adjclose.head()

Tickers,WIKI/AAPL,WIKI/MSFT,WIKI/GOOGL,WIKI/AMZN,WIKI/BRK_B,WIKI/JNJ,WIKI/UNH,WIKI/XOM,WIKI/V,WIKI/JPM,WIKI/WMT,WIKI/PG,WIKI/NVDA,WIKI/TSLA,WIKI/LLY,WIKI/CVX,WIKI/MA,WIKI/HD,WIKI/MRK,WIKI/PFE
Date,,,,,,,,,,,,,,,,,,,,
2013-01-02,71.195748,24.194478,362.744758,257.3100,93.2000,61.595109,50.497133,75.506133,37.507891,39.388479,61.097979,59.268462,11.879926,35.36,42.997486,91.769514,49.423019,57.369885,35.464703,21.715242
2013-01-03,70.296565,23.870367,362.955409,258.4800,93.6211,61.508159,48.136156,75.369948,37.536859,39.309103,60.709719,58.892642,11.889266,34.77,43.179863,91.378794,49.493724,57.207211,36.314003,21.664955
2013-01-04,68.338996,23.423618,370.127548,259.1500,93.8500,62.212451,48.228743,75.718922,37.843430,40.005853,60.939145,59.012221,12.281528,34.40,44.777830,91.860960,49.491787,57.098761,36.005167,21.757147
2013-01-07,67.937002,23.379820,368.512563,268.4592,93.4500,62.082027,48.228743,74.842231,38.113792,40.049952,60.356755,58.610777,11.926624,34.34,44.717038,91.237469,50.348968,56.792571,36.133848,21.773909
2013-01-08,68.119845,23.257183,367.785318,266.3800,93.8100,62.090722,47.589890,75.310367,38.468642,40.129328,60.524413,58.516823,11.666517,33.68,45.055738,90.830122,50.183248,57.134911,36.185321,21.807433


In [152]:
#making a dataframe of returns based on adj. close
returns = pd.DataFrame()
for x in tickers:
    returns[x+' Returns'] = df[x]['Adj. Close'].pct_change()[1:]
returns.head()

,WIKI/AAPL Returns,WIKI/MSFT Returns,WIKI/GOOGL Returns,WIKI/AMZN Returns,WIKI/BRK_B Returns,WIKI/JNJ Returns,WIKI/UNH Returns,WIKI/XOM Returns,WIKI/V Returns,WIKI/JPM Returns,WIKI/WMT Returns,WIKI/PG Returns,WIKI/NVDA Returns,WIKI/TSLA Returns,WIKI/LLY Returns,WIKI/CVX Returns,WIKI/MA Returns,WIKI/HD Returns,WIKI/MRK Returns,WIKI/PFE Returns
Date,,,,,,,,,,,,,,,,,,,,
2013-01-03,-0.012630,-0.013396,0.000581,0.004547,0.004518,-0.001412,-0.046755,-0.001804,0.000772,-0.002015,-0.006355,-0.006341,0.000786,-0.016686,0.004242,-0.004258,0.001431,-0.002836,0.023948,-0.002316
2013-01-04,-0.027847,-0.018716,0.019760,0.002592,0.002445,0.011450,0.001923,0.004630,0.008167,0.017725,0.003779,0.002030,0.032993,-0.010641,0.037007,0.005277,-0.000039,-0.001896,-0.008505,0.004255
2013-01-07,-0.005882,-0.001870,-0.004363,0.035922,-0.004262,-0.002096,0.000000,-0.011578,0.007144,0.001102,-0.009557,-0.006803,-0.028897,-0.001744,-0.001358,-0.006787,0.017320,-0.005362,0.003574,0.000770
2013-01-08,0.002691,-0.005245,-0.001973,-0.007745,0.003852,0.000140,-0.013246,0.006255,0.009310,0.001982,0.002778,-0.001603,-0.021809,-0.019220,0.007574,-0.004465,-0.003291,0.006028,0.001425,0.001540
2013-01-09,-0.015629,0.005650,0.006570,-0.000113,-0.005223,0.004481,0.018872,-0.003843,0.015248,-0.000659,-0.000292,0.005401,-0.022535,-0.001188,0.021010,0.002563,0.028137,-0.000791,0.009720,0.017294


In [154]:
# log daily return...better measure than arithmetic returns
log_ret = np.log(adjclose/adjclose.shift(1))[1:]
log_ret.head()

Tickers,WIKI/AAPL,WIKI/MSFT,WIKI/GOOGL,WIKI/AMZN,WIKI/BRK_B,WIKI/JNJ,WIKI/UNH,WIKI/XOM,WIKI/V,WIKI/JPM,WIKI/WMT,WIKI/PG,WIKI/NVDA,WIKI/TSLA,WIKI/LLY,WIKI/CVX,WIKI/MA,WIKI/HD,WIKI/MRK,WIKI/PFE
Date,,,,,,,,,,,,,,,,,,,,
2013-01-03,-0.012710,-0.013487,0.000581,0.004537,0.004508,-0.001413,-0.047883,-0.001805,0.000772,-0.002017,-0.006375,-0.006361,0.000786,-0.016826,0.004233,-0.004267,0.001430,-0.002840,0.023666,-0.002318
2013-01-04,-0.028242,-0.018893,0.019568,0.002589,0.002442,0.011385,0.001922,0.004619,0.008134,0.017570,0.003772,0.002028,0.032460,-0.010698,0.036339,0.005263,-0.000039,-0.001898,-0.008541,0.004246
2013-01-07,-0.005900,-0.001872,-0.004373,0.035292,-0.004271,-0.002099,0.000000,-0.011646,0.007119,0.001102,-0.009603,-0.006826,-0.029323,-0.001746,-0.001359,-0.006810,0.017171,-0.005377,0.003568,0.000770
2013-01-08,0.002688,-0.005259,-0.001975,-0.007775,0.003845,0.000140,-0.013335,0.006235,0.009267,0.001980,0.002774,-0.001604,-0.022050,-0.019407,0.007546,-0.004475,-0.003297,0.006010,0.001423,0.001538
2013-01-09,-0.015752,0.005634,0.006549,-0.000113,-0.005237,0.004471,0.018696,-0.003850,0.015133,-0.000660,-0.000292,0.005386,-0.022793,-0.001188,0.020792,0.002559,0.027749,-0.000791,0.009673,0.017147


### Defining the functions needed for optimizing the portfolio, as well as the constraints and objective

In [157]:
#define function that will return: returns, volatility, and sharpe ratio
def get_ret_vol_sharper(weights): 
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [158]:
# define function that will return the negative sharpe ratio
def neg_sharpe(weights): 
    return get_ret_vol_sharper(weights)[2] * -1

In [159]:
# check allocation sums to 1
def check_sum(weights): 
    return np.sum(weights) - 1

In [160]:
# create constraint variable
cons = ({'type':'eq','fun':check_sum})

In [161]:
# create weight boundaries
bounds = (())
for x in range(len(tickers)):
    bounds = bounds + ((0,1),)
bounds

((0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1))

In [162]:
#creating an initial guess array...simply an even weight distribution
init_guess = []
for x in range(len(tickers)):
    p = 1/len(tickers)
    init_guess = init_guess + [[p, ]]
init_guess

[[0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05],
 [0.05]]

### Results

In [163]:
#we store the results of the minimization function in a results variable
#set the method = SLSQP --> (Sequential Least SQuares Programming optimizer)
results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
results

C:\Users\chait\AppData\Local\Temp\ipykernel_1868\2936979482.py:3: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)


     fun: -2.0373989573427105
     jac: array([ 2.29596049e-01, -3.18735838e-04,  2.92430669e-01,  6.15298748e-04,
        2.10017502e-01, -2.49743462e-05,  7.00443983e-04,  8.08190495e-01,
        5.40177226e-02,  1.75961137e-01,  2.76695043e-01,  2.78102815e-01,
       -1.34408474e-03,  2.06232071e-04,  2.49986798e-01,  6.74991041e-01,
        8.50515962e-02,  3.62128019e-04,  3.11456442e-01,  3.49497914e-01])
 message: 'Optimization terminated successfully'
    nfev: 235
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([0.00000000e+00, 1.23896698e-02, 0.00000000e+00, 1.27191047e-01,
       0.00000000e+00, 2.49005051e-02, 3.34077882e-01, 6.97783183e-17,
       5.68773729e-17, 0.00000000e+00, 7.88190130e-18, 3.35060360e-17,
       2.83645563e-01, 4.21127735e-02, 5.83225281e-17, 2.15986359e-16,
       0.00000000e+00, 1.75682560e-01, 0.00000000e+00, 5.79840530e-18])

In [164]:
#shows the weights of the portfolio for optimal sharpe ratio
results.x

array([0.00000000e+00, 1.23896698e-02, 0.00000000e+00, 1.27191047e-01,
       0.00000000e+00, 2.49005051e-02, 3.34077882e-01, 6.97783183e-17,
       5.68773729e-17, 0.00000000e+00, 7.88190130e-18, 3.35060360e-17,
       2.83645563e-01, 4.21127735e-02, 5.83225281e-17, 2.15986359e-16,
       0.00000000e+00, 1.75682560e-01, 0.00000000e+00, 5.79840530e-18])

In [165]:
tickers[2].split('/')[1]

'GOOGL'

In [ ]:
#removing 'WIKI/' from tickers
stocks=[]
for x in range(len(tickers)):
    stocks.append(tickers[x].split('/')[1])

#turning results.x into percents
percent = pd.Series(["{0:.2f}%".format(val * 100) for val in results.x],
                    index = range(len(results.x)))

### Optimized Portfolio

In [166]:
#making a dataframe with stocks and associated returns
results_dict = {'Stock' : stocks,
              'Portfolio Weight(as percent)' : percent}
results_df = pd.DataFrame(results_dict)
results_df

,Stock,Portfolio Weight(as percent)
0,AAPL,0.00%
1,MSFT,1.24%
2,GOOGL,0.00%
3,AMZN,12.72%
4,BRK_B,0.00%
5,JNJ,2.49%
6,UNH,33.41%
7,XOM,0.00%
8,V,0.00%
9,JPM,0.00%


In [167]:
#will return the sharpe ratio of the portfolio
get_ret_vol_sharper(results.x)[2]

#the below code also works
#neg_sharpe(results.x)*-1

2.0373989573427105

In [168]:
#will return the volatility of the portfolio
get_ret_vol_sharper(results.x)[1]

0.17522974583924458

In [169]:
#will get the expected returns for the portfolio
get_ret_vol_sharper(results.x)[0] #35.7% returns

0.3570129014683051